# 要約 
このJupyter Notebookでは、Kaggleの「20の質問」ゲームコンペティションにおけるエージェントの実装が行われています。具体的には、質問を行い、推測をするエージェント（質問者）と、それに応じて「はい」または「いいえ」と回答するエージェント（回答者）の機能が実装されています。

ノートブックは以下の内容に取り組んでいます：

1. **ライブラリのインポート**: NumPyとPandasを使用して基本的なデータ操作を行います。
2. **ディレクトリ管理**: 提出物用のディレクトリを作成し、そこに必要なファイルを格納します。
3. **CSVファイルの作成とデータ書き込み**: 動物のリストを含むCSVファイルを作成し、エージェントが推測に使用するデータを用意します。
4. **エージェントのロジック実装**: Pythonで書かれたエージェントのメインロジックが定義されています。質問者は、ランダムに選択された動物についての質問を行い、回答者はそれに対して「はい」または「いいえ」で応答します。エージェントの動作は、ゲームのターンに応じて異なるレスポンスを生成します。
5. **ファイルの圧縮と提出準備**: 最後に、作成した提出物をtar.gz形式で圧縮し、提出の準備を行っています。

使用されている主なライブラリは、`numpy`、`pandas`、および`csv`で、これらはデータ操作とファイル入出力のために利用されています。全体として、このノートブックは、コンペティションルールに従ったエージェントの構築と、そのためのデータ管理を行っていることが特徴です。

---


# 用語概説 
以下に、Jupyter Notebook内で使われている専門用語のリストとその簡単な解説を示します。初心者がつまずきやすいマイナーな用語や特有のドメイン知識に焦点を当てています。

1. **Kaggle**: 機械学習の競技やデータ分析のプラットフォームで、データセットを共有したり、コンペティションを通じてスキルを磨くことができる。

2. **Dockerイメージ**: アプリケーションをコンテナとして実行するためのテンプレート。指定されたアプリケーションの環境を包括的に構成する。具体的には、アプリが動作するために必要なすべてのライブラリや設定が含まれています。

3. **CSV (Comma-Separated Values)**: データをカンマで区切った形式で保存するファイルフォーマット。データフレームをファイルに保存したり、読み込みたりするのに非常に一般的です。

4. **エージェント**: コンペティションにおける参加者やプログラムのこと。質問をする役割や回答をする役割を持つことがあります。

5. **libディレクトリ**: 「ライブラリ」の略として使われることが多く、他のスクリプトやデータを格納するための専用のフォルダ。

6. **sys.path**: Pythonのモジュール検索パスを格納したリスト。このリストにパスを追加することで、新しいモジュールをインポートできるようになる。

7. **obs (observation)**: エージェントが環境から受け取る情報のこと。コンペティションでは、これが他のプレイヤーから得られる質問や回答などの情報を指します。

8. **response**: エージェントが生成する応答のこと。質問者や回答者としての役割に応じて、異なる内容を返します。

9. **turnType**: ゲーム内のエージェントの現在の役割や行動のタイプを示します。"ask"は質問をする、"guess"は推測をする、"answer"は回答することを意味します。

10. **random.choice()**: Pythonのrandomモジュールの関数で、与えられたリストの中からランダムに1つの要素を選ぶために使用されます。ゲーム戦略においてランダム性を導入します。

11. **tar**: Unix系のオペレーティングシステムで使用されるファイルアーカイブコマンド。複数のファイルを1つのアーカイブファイル（tarファイル）にまとめます。

12. **pigz**: プロセス間で圧縮するための高速な圧縮ツールで、gzipよりも速く処理を行うことができます。

13. **pv**: パイプの進捗状況を表示するためにツール。パイプライン処理の進行状況を監視するために使われます。

これらの用語に関して知識を深めることで、Jupyter Notebookの内容や機械学習プロジェクトへの理解が一層深まるでしょう。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下は読み込むのに便利なパッケージのいくつかです

import numpy as np # 線形代数のためのNumPyライブラリをインポートします
import pandas as pd # データ処理、CSVファイル入出力のためのPandasライブラリをインポートします

# 入力データファイルは読み取り専用の"../input/"ディレクトリ内にあります
# 例えば、これを実行すると（クリックするか、Shift+Enterを押す）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを印刷します

# 現在のディレクトリ（/kaggle/working/）には最大20GBまで書き込むことができ、このディレクトリに作成したバージョンを使用して"Save & Run All"で出力が保存されます
# 一時ファイルは/kaggle/temp/に書き込むことができますが、そのファイルは現在のセッション外には保存されません

In [ ]:
import os
submission_directory = "/kaggle/working/submission" # 提出物を保存するためのメインディレクトリのパスを設定
submission_subdirectory = "lib" # サブディレクトリの名前を設定

# メインディレクトリが存在しない場合は作成します
if not os.path.exists(submission_directory):
    os.mkdir(submission_directory) # メインディレクトリを作成
    subdirectory_path = os.path.join(submission_directory, submission_subdirectory) # サブディレクトリのフルパスを生成
    os.mkdir(subdirectory_path) # サブディレクトリを作成

In [ ]:
import csv
# サブディレクトリ内に"example.csv"という名前のファイルを作成し、書き込みモードで開きます
with open(os.path.join(subdirectory_path, "example.csv"), mode='w') as file:
    writer = csv.writer(file) # CSVファイルに書き込むためのwriterオブジェクトを作成
    writer.writerow(["cow", "horse"]) # CSVファイルの最初の行に"cow"と"horse"のデータを書き込みます

In [ ]:
%%writefile /kaggle/working/submission/main.py

import os
import sys
import csv
import random

# 提出物/libディレクトリに他のファイル（例：モデルの重み）を置く場合、パスを設定する必要があります
KAGGLE_COMPETITION_PATH = "/kaggle_simulations/agent/" # コンペティションパスを定義
if os.path.exists(KAGGLE_COMPETITION_PATH):  # コンペティション内で実行している場合
    subdirectory_path = os.path.join(KAGGLE_COMPETITION_PATH, "lib") # libフォルダへのパスを設定
else: # ノートブックで実行している場合
    subdirectory_path = os.path.join("/kaggle/working/submission/", "lib") # libフォルダへのパスを設定
sys.path.insert(0, subdirectory_path) # パスをシステムパスに追加します

# "example.csv"ファイルを読み込みモードで開きます
with open(os.path.join(subdirectory_path, "example.csv"), mode='r') as file:
    reader = csv.reader(file) # CSVファイルを読み込むためのreaderオブジェクトを作成
    guess_list = list(reader) # ファイルの全行をリストに変換します
    guess_list = guess_list[0] # 1行目のリストを取得します

animal = random.choice(guess_list) # guess_listからランダムに動物を選択します

def agent_fn(obs, cfg):
    
    # エージェントが質問者で、turnTypeが"ask"の場合
    if obs.turnType == "ask":
        response = f'Does it look like a {animal}?' # 選ばれた動物について尋ねます
    # エージェントが質問者で、turnTypeが"guess"の場合
    elif obs.turnType == "guess":
        if obs.answers[-1] == "yes": # 最後の回答が"yes"の場合
            response = animal # 選択した動物を答えます
        else:
            response = "penguin" # 選択した動物が否定された場合の応答
    # エージェントが回答者の場合
    elif obs.turnType == "answer":
        if obs.keyword in obs.questions[-1]: # 最後の質問にキーワードが含まれている場合
            response = "yes" # "yes"と返答します
        else:
            response = "no" # "no"と返答します
        
    return response # 生成した応答を返します

In [ ]:
!apt install pigz pv > /dev/null # pigzとpvをインストールします。出力は表示しません
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . # submissionディレクトリを圧縮してsubmission.tar.gzを作成します